### 1. LLMs (Large Language Models)
    •Wraps models like OpenAI, Anthropic, Hugging Face, etc.
    •Provides standardized interfaces and API calls.


In [34]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")

## 2.Prompts
    •Templates that dynamically construct queries for LLMs.
    •Can include variables ({input}) and instructions.


In [35]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {company_name} tagline for this.I do transport some machine tools.")],
)

In [ ]:
prompt(any input variables replace dynamically ) -> llm - > JsonOutputParser()

### 3.Chains
    •Combine LLM + Prompt + other components into a workflow.
    •Most common: LLMChain

In [36]:
from langchain.chains import LLMChain
legacy_chain = LLMChain(llm =llm, prompt=prompt)


In [37]:
legacy_result = legacy_chain({"company_name": "sea tools"})

In [38]:
legacy_result

{'company_name': 'sea tools',
 'text': '"Seamlessly Navigating Your Machine Tools Across Waters."'}

In [6]:
legacy_chain.run("funny")

/var/folders/91/s6b95dhx7q33x589x_2pm1x40000gn/T/ipykernel_71300/814955422.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  legacy_chain.run("funny")


'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

In [39]:
legacy_chain.invoke("Planets")

{'company_name': 'Planets',
 'text': '"Planets: Your Trusted Orbit for Precision Tool Transport."'}

### 4. Tools
    •External functions the LLM can call (e.g., search, calculator, API).
    •Used by agents.


In [51]:
from langchain.tools import Tool

def get_weather(location: str):
#     try to add weather api or any other api for calling. realtime weather
    
    return "Sunny in " + location

tool = Tool(name="WeatherTool", func=get_weather, description="Gets weather info.")

def get_realtime_events(input:str):
#     any api access
    return "Donal Trump"

real_tool = Tool(name="RealEventsTool", func=get_realtime_events, description="Gets real events info.")


### 5.Agents
    •Decision-makers that use LLMs to choose tools and actions.
    •Powerful for building autonomous workflows (e.g., “search → summarize → email”).


In [50]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[tool,real_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Who won the elections in 2024 USA?")




> Entering new AgentExecutor chain...
To answer the question about who won the 2024 USA elections, I need to check for real-time events or updates related to the elections. 

Action: RealEventsTool
Action Input: 2024 USA elections results
Observation: Donal Trump
Thought:I now know the final answer.

Final Answer: Donald Trump won the 2024 USA elections.

> Finished chain.


'Donald Trump won the 2024 USA elections.'

In [ ]:
ReAct Framwork(which is nothing but prompts) = Reasoning + Acting

In [ ]:


think or make plan === step1
                        step2
                        step3
action -step1
observe -output of step1

In [ ]:
ReAct ==== you will get a prompt you need to make steps very carefully and execute one step at a time 
and observe at the same time and if you think you have answer stop it or terminate it.

agent is doing making the prompt: give me stept to how to acheive this going to boston - >LLMs ->
    you can make reservation flight ticket
    you can book a ticket amtrack 
    you can book a ticket for greyhound
    
1)you can make reservation flight ticket - >llm - > 
2)

### 6.Document Loaders
    •Load text data from PDFs, CSVs, Notion, websites, etc.


In [52]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/Users/rams/Downloads/The Staff Engineers Path.pdf")
docs = loader.load()


### 7.Text Splitters
    •Break long documents into chunks for LLM processing (important for RAG).


In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)


In [54]:
chunks[:10]

[Document(metadata={'producer': 'calibre 3.32.0 [https://calibre-ebook.com]', 'creator': 'calibre 3.32.0 [https://calibre-ebook.com]', 'creationdate': '2025-06-24T12:00:04+00:00', 'author': 'Reilly, Tanya;', 'title': "The Staff Engineer's Path", 'source': '/Users/rams/Downloads/The Staff Engineers Path.pdf', 'total_pages': 439, 'page': 1, 'page_label': '2'}, page_content='Praise\tfor\t\nThe\tStaff\tEngineer’s\tPath\nThe\tbook\tI\twish\tI’d\thad\twhen\tI\tstepped\tup\tto\tprincipal\tengineer.\tIf\tyou\tare\nwondering\twhat\tStaff+\tmeans,\tand\thow\tto\tbe\tsuccessful\tin\tthe\trole\tin\tyour\norganization,\tTanya\thas\tlaid\tout\tthe\tpath,\twith\tlots\tof\tpractical,\tinsightful\nadvice\tfor\tyou.\tThis\tbook\twill\tprovide\tthe\ttools\tto\tenable\tyou\tto\tthrive\t\nin\ta\ntech\ttrack\n\trole,\tacting\twith\tinfluence\tand\timpact.\n—\nSarah\tWells,\tindependent\tconsultant\tand\tauthor,\t\nformer\tprincipal\tengineer\tat\tthe\t\nFinancial\tTimes'),
 Document(metadata={'producer': 'c

### 8.Vector Stores
    •Store document embeddings for semantic search (FAISS, Chroma, Pinecone, Weaviate).


In [20]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

store = FAISS.from_documents(chunks, OpenAIEmbeddings(model='text-embedding-3-small'))
retrieved = store.similarity_search("LangChain architecture")


In [19]:
# !pip install faiss-cpu

### 9.Retrievers
    •Abstraction over vector store or other sources.
    •Used in RAG pipelines to fetch relevant context.


In [ ]:
retriever = store.as_retriever()
retriever.get_relevant_documents("How to use agents?")


### 10.Memory
    •Maintains conversation state across turns.


In [55]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)


/var/folders/91/s6b95dhx7q33x589x_2pm1x40000gn/T/ipykernel_71300/1023760234.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [56]:
chain.invoke("kid")

{'company_name': 'kid', 'history': '', 'text': '"Hauling the Cool Tools!"'}

In [58]:
chain.invoke("Bus")

{'company_name': 'Bus',
 'history': 'Human: kid\nAI: "Hauling the Cool Tools!"\nHuman: what kid tageline you know ?\nAI: Creating a catchy and kid-friendly tagline for transporting machine tools can be a fun way to make the concept more engaging. Here\'s a suggestion:\n\n"Zoomin\' with Machine Movers Magic!" \n\nThis tagline combines a sense of movement and excitement ("Zoomin\'") with a whimsical twist ("Magic") to capture children\'s attention and imagination.',
 'text': '"Driving Precision: Transporting Your Tools with Care and Reliability"'}

In [59]:
chain.invoke("tell me what do you in past ?")

{'company_name': 'tell me what do you in past ?',
 'history': 'Human: kid\nAI: "Hauling the Cool Tools!"\nHuman: what kid tageline you know ?\nAI: Creating a catchy and kid-friendly tagline for transporting machine tools can be a fun way to make the concept more engaging. Here\'s a suggestion:\n\n"Zoomin\' with Machine Movers Magic!" \n\nThis tagline combines a sense of movement and excitement ("Zoomin\'") with a whimsical twist ("Magic") to capture children\'s attention and imagination.\nHuman: Bus\nAI: "Driving Precision: Transporting Your Tools with Care and Reliability"',
 'text': '"Delivering Precision, Transporting Excellence."'}

In [ ]:
from langchain_core.output_parsers import StrOutputParser